In [ ]:
%%capture
!pip install https://github.com/OlivierBeq/Papyrus-scripts/tarball/master
!pip install -U kaleido
get_ipython().kernel.do_shutdown(True)

In [ ]:
!gdown "https://drive.google.com/uc?id=1A7XDndNywq_g6g0gdy3LR8PGvjXgYf_Y&confirm=t"
!gdown "https://drive.google.com/uc?id=15zbhi5S8e-xFe-V3KJhqPPaMCz00dVrV&confirm=t"
!gdown "https://drive.google.com/uc?id=1d4zjI8FGIYi6hQkigNKs9bxRGrQgwLp9&confirm=t"

Downloading...
From: https://drive.google.com/uc?id=1A7XDndNywq_g6g0gdy3LR8PGvjXgYf_Y&confirm=t
To: /content/05.5_combined_set_protein_targets.tsv.xz
100% 1.71M/1.71M [00:00<00:00, 58.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1KHjl5uZ8cBjUg50IKbBf8qp07gglqVoh&confirm=t
To: /content/05.5_combined_set_without_stereochemistry.tsv.xz
100% 719M/719M [00:05<00:00, 126MB/s]
Downloading...
From: https://drive.google.com/uc?id=1d4zjI8FGIYi6hQkigNKs9bxRGrQgwLp9&confirm=t
To: /content/05.5_combined_set_with_stereochemistry.tsv.xz
100% 690M/690M [00:07<00:00, 96.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=14y3R45-t2pA-_Vkj5RiG1dPu5pOb4T72&confirm=t
To: /content/05.5_combined_set_without_stereochemistry_fixed.tsv.gz
100% 1.80G/1.80G [00:16<00:00, 112MB/s]


In [ ]:
from collections import Counter, defaultdict
import pandas as pd
from tqdm.auto import tqdm

from papyrus_scripts.reader import read_protein_set

In [ ]:
Counter(pd.read_csv('05.5_combined_set_without_stereochemistry_fixed.tsv.xz', sep='\t', usecols=['Quality']).sqeez)

In [ ]:
pbar = tqdm(total=60)
stats = {}
protein_data = pd.read_csv('05.5_combined_set_protein_targets.tsv.xz', sep='\t', keep_default_na=False)
for chunk in pd.read_csv('05.5_combined_set_without_stereochemistry_fixed.tsv.gz', sep='\t', chunksize=1000000):
    merged = chunk.merge(protein_data, on='target_id')
    for species in tqdm(merged['Organism'].unique(), leave=False):
        high_qual = merged[(merged['Organism'] == species) & (merged['Quality'] == 'High')]
        med_qual = merged[(merged['Organism'] == species) & (merged['Quality'] == 'Medium')]
        low_qual = merged[(merged['Organism'] == species) & (merged['Quality'] == 'Low')]
        if species in stats:
            stats[species]['High'] += len(high_qual.index)
            stats[species]['Medium'] += len(med_qual.index)
            stats[species]['Low'] += len(low_qual.index)
            stats[species]['Total'] += len(high_qual.index) + len(med_qual.index) + len(low_qual.index)
        else:
            stats[species] = {'High': len(high_qual.index), 'Medium': len(med_qual.index), 'Low': len(low_qual.index), 'Total': len(high_qual.index) + len(med_qual.index) + len(low_qual.index)}
    pbar.update()

In [ ]:
pd.DataFrame([{'Organism': key, **value} for key, value in stats.items()]).sort_values(by='Total', ascending=False).iloc[:10,:].reset_index(drop=True)

,Organism,High,Medium,Low,Total
0,Homo sapiens (Human),987436,246401,54364908,55598745
1,Mus musculus (Mouse),42078,6682,2465157,2513917
2,Rattus norvegicus (Rat),60475,32061,1151955,1244491
3,Escherichia coli (strain K12),539,11298,60030,71867
4,Equus caballus (Horse),18330,32,27988,46350
5,Influenza A virus (A/WSN/1933(H1N1)),23813,0,9143,32956
6,Trypanosoma cruzi,5935,30,23927,29892
7,Schistosoma mansoni (Blood fluke),13916,0,14473,28389
8,Bacillus subtilis,12106,0,11693,23799
9,Bos taurus (Bovine),5944,5105,8918,19967


In [ ]:
pd.DataFrame([{'Organism': key, **value} for key, value in stats.items()]).sort_values(by='Total', ascending=False).reset_index(drop=True)

,Organism,High,Medium,Low,Total
0,Homo sapiens (Human),987436,246401,54364908,55598745
1,Mus musculus (Mouse),42078,6682,2465157,2513917
2,Rattus norvegicus (Rat),60475,32061,1151955,1244491
3,Escherichia coli (strain K12),539,11298,60030,71867
4,Equus caballus (Horse),18330,32,27988,46350
...,...,...,...,...,...
494,Komagataella pastoris (Yeast) (Pichia pastoris),0,0,1,1
495,Phytophthora sojae (Soybean stem and root rot ...,1,0,0,1
496,Chondrostereum purpureum,1,0,0,1
497,Duck hepatitis B virus (strain Germany/DHBV-3)...,0,1,0,1


In [ ]:
pbar = tqdm(total=60)
types = defaultdict(int)
for chunk in pd.read_csv('05.5_combined_set_without_stereochemistry_fixed.tsv.gz', sep='\t', chunksize=1000000, usecols=['type_IC50', 'type_EC50', 'type_KD', 'type_Ki', 'type_other']):
    types['IC50'] += chunk['type_IC50'].fillna(0).astype(str).str.split(';').map(lambda x: sum(int(k) for k in x)).sum()
    types['EC50'] += chunk['type_EC50'].fillna(0).astype(str).str.split(';').map(lambda x: sum(int(k) for k in x)).sum()
    types['KI'] += chunk['type_Ki'].fillna(0).astype(str).str.split(';').map(lambda x: sum(int(k) for k in x)).sum()
    types['KD'] += chunk['type_KD'].fillna(0).astype(str).str.split(';').map(lambda x: sum(int(k) for k in x)).sum()
    types['Other'] += chunk['type_other'].fillna(0).astype(str).str.split(';').map(lambda x: sum(int(k) for k in x)).sum()
    pbar.update()
pbar.close()

  0%|          | 0/60 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (17,18,19,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [ ]:
types

defaultdict(int,
            {'EC50': 142251,
             'IC50': 1082403,
             'KD': 119455,
             'KI': 509022,
             'Other': 58314761})